In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Sentiment_Analysis

/content/drive/MyDrive/Sentiment_Analysis


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from matplotlib import pyplot as plt
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import unicodedata

In [ ]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install language-check
! pip install pycontractions

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm
  libnss-mdns fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86dga1 open

In [ ]:
pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=5d27c4fed9615fffd4da914ade7e0d454ba6b95670900af67ad444030628959e
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def clean_text_data(data: pd.Series) -> pd.Series:
  url = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
  assert data.isna().sum() == 0, data[data.isna()]
  data = data.map(lambda x:str(x).lower()) #lower case
  data = data.map(lambda x:re.sub(r"\b[^\s]+@[^\s]+[.][^\s]+\b", "", x)) #email
  data = data.map(lambda x:re.sub(url, "", x)) #url
  data = data.map(lambda x:re.sub(r'[^a-zA-z.,!?/:;\"\'\s]', "", x)) #numbers
  data = data.map(lambda x:re.sub(r'^\s*|\s\s*', ' ', x).strip()) #white space
  data = data.map(lambda x:''.join([c for c in x if c not in string.punctuation])) #punctuations
  data = data.map(lambda x:re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s]', '', x)) #special char
  data = data.map(lambda x:unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')) #unicode
  tokenizer = RegexpTokenizer(r'\w+')
  data = data.map(lambda x:tokenizer.tokenize(x))  #remove punctuation and tokenize
  return data


In [ ]:
train_df = pd.read_csv("data/train.csv")
print(train_df.shape)
train_df.head()

(27481, 4)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
test_df = pd.read_csv("data/test.csv")
print(test_df.shape)
test_df.head()

(3534, 3)


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
# process data
train_df.dropna(inplace=True)
train_df.text = clean_text_data(train_df.text)
train_df.selected_text = clean_text_data(train_df.selected_text)
test_df.text = clean_text_data(test_df.text)

In [ ]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",neutral
1,549e992a42,"[sooo, sad, i, will, miss, you, here, in, san,...","[sooo, sad]",negative
2,088c60f138,"[my, boss, is, bullying, me]","[bullying, me]",negative
3,9642c003ef,"[what, interview, leave, me, alone]","[leave, me, alone]",negative
4,358bd9e861,"[sons, of, why, couldnt, they, put, them, on, ...","[sons, of]",negative


In [ ]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,"[last, session, of, the, day]",neutral
1,96d74cb729,"[shanghai, is, also, really, exciting, precise...",positive
2,eee518ae67,"[recession, hit, veronique, branquinho, she, h...",negative
3,01082688c6,"[happy, bday]",positive
4,33987a8ee5,"[i, like, it]",positive


In [ ]:
# #to use spellcheck
# from autocorrect import Speller
# spell = Speller(lang="en")
# df.input = df.input.map(lambda x:tokenizer.tokenize(x))  #remove punctuation and tokenize
# df.input = df.input.map(lambda x:[spell(i) for i in x])  #spell check

In [ ]:
# save data
train_df.to_csv("data/preprocessed_train.csv",index=False)
test_df.to_csv("data/preprocessed_test.csv",index=False)

In [ ]:
# process the twitter data
column_names = ['score', 'id', 'time', 'action', 'user', 'text']  # replace with actual column names
twitter_df = pd.read_csv("data/first_100000_rows.csv", header=None, names=column_names)
print(twitter_df.head())


   score          id                          time    action             user  \
0      0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1      0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2      0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3      0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4      0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   

                                                text  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  is upset that he can't update his Facebook by ...  
2  @Kenichan I dived many times for the ball. Man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  
(100001, 6)


In [ ]:
# process data
twitter_df.dropna(inplace=True)
twitter_df.text = clean_text_data(twitter_df.text)

In [ ]:
twitter_df.head()

,score,id,time,action,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"[switchfoot, awww, thats, a, bummer, you, shou..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"[is, upset, that, he, cant, update, his, faceb..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,"[kenichan, i, dived, many, times, for, the, ba..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,"[my, whole, body, feels, itchy, and, like, its..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"[nationwideclass, no, its, not, behaving, at, ..."


In [ ]:
# save data
twitter_df.to_csv("data/first_100000_rows_processed.csv",index=False)